In [9]:
def Notch_Fliege(f0, Q = 10, margin = 2, RQ = None, R0 = None, C0 = None, cap = None, res = None, visible='Off'):
    ### Required libraries are imported
    import si_prefix
    import numpy as np
    from tabulate import tabulate
    
    #
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]
    
    ### Initialize capacitor values
    if cap is None:
        # Default capacitor values
        cap_p = np.array([0.5, 0.75, 1, 1.2, 1.3, 1.5, 1.6, 1.8, 2, 2.2, 2.4, 2.5, 2.7, 3, 3.3, 3.6, 3.9, 4.3, 4.7, \
         5, 6, 6.2, 6.8, 7, 7.5, 8, 8.2, 9, 10, 11, 12, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 39,\
         43, 47, 51, 56, 62, 68, 82, 100, 110, 120, 150, 180, 200, 220, 330, 470, 680, 820])*(10**-12)

        cap_n = np.array([1, 1.5, 2.2, 3.3, 4.7, 5.6, 6.8, 8.2, 10, 15, 22, 33, 47, 68, 82, 100, 220, 470, 680, 820])\
                         *(10**-9)

        cap_u = np.array([1, 1.5, 2.2, 3.3, 4.7, 5.6])*(10**-6)

        cap_library = np.concatenate((cap_p,cap_n,cap_u),axis=0)
        
    elif type(cap) is np.ndarray:
        # Assign custom capacitor values
        cap_library = cap
        
    else:
        raise TypeError("Capacitor values must be np.ndarray data type!")
    

    ### Initialize resistor values
    if res is None:
        # Default resistor values
        res_0 = np.array([1, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6,\
                          3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1])
        res_1 = res_0*10
        res_2 = res_1*10
        res_3 = res_2*10
        res_4 = res_3*10
        res_5 = res_4*10
        res_6 = res_5*10
        res_library = np.concatenate(([0],res_0,res_1,res_2,res_3,res_4,res_5,res_6,[10**7]),axis=0)
        
    elif type(res) is np.ndarray:
        # Assign custom resistor values
        res_library = res
    
    else:
        raise TypeError("Resistor values must be np.ndarray data type!")
    
    
    ### Q value 
    Q_ideal = Q
    Q_margin = (margin/100)*Q_ideal
    
    ### Assign cut-off frequency
    f0_ideal = f0
    
    ### Start R and C value estimation
    R0C0 = 2*np.pi*f0_ideal
    C0_value, C2_value, C3_value = [], [], []
    R0_value, RQ_value = [], []
    Q_value, Gain_value, f0_value = [], [], []
    for c_temp in cap_library:
        # Start assigning C0 capacitor value
        C0_candid = c_temp
        
        # Find the closest C0 values from capacitor library
        simple_C0 = si_prefix.split(C0_candid)
        if (simple_C0[1] == -6 and simple_C0[0]>= 6) or simple_C0[1] > -6:
            continue
        else:
            C0_candid = find_nearest(cap_library,C0_candid)
        
        # Estimate ideal R0
        R0_ideal = R0C0 / C0_candid
        # RQ_ideal = 2 * R0_ideal * Q_ideal
        # Find the closest R0 values from resistor library
        simple_R0 = si_prefix.split(R0_ideal)
        if (simple_R0[1] == 6 and simple_R0[0] >= 10) or simple_R0[1] > 6:
            continue
        else:
            R0_candid_base = np.around(np.array(simple_R0[0]),decimals=1)
            R0_candid_power = (10**simple_R0[1])
            R0_candid = find_nearest(res_library,R0_candid_base*R0_candid_power)

            RQ_ideal = 2 * R0_candid * Q_ideal
            simple_RQ = si_prefix.split(RQ_ideal)
            if (simple_RQ[1] == 6 and simple_RQ[0] >= 10) or simple_RQ[1] > 6:
                continue
            else:
                RQ_candid_base = np.around(np.array(simple_RQ[0]),decimals=1)
                RQ_candid_power = (10**simple_RQ[1])        
                RQ_candid = find_nearest(res_library,RQ_candid_base*RQ_candid_power)

            fc_candid = 1/(2 * np.pi * R0_candid * C0_candid)
            Q_candid = RQ_candid/(2 * R0_candid)

            # Check if actual Q values is between decided margin values
            if np.abs(Q_candid-Q_ideal) <= Q_margin:
                # Append capacitor and resistor values to empty array
                C0_value.append(si_prefix.si_format(C0_candid))
                R0_value.append(si_prefix.si_format(R0_candid))
                RQ_value.append(si_prefix.si_format(RQ_candid))
                
                Q_value.append(Q_candid)
                f0_value.append(fc_candid)
    
    ### Create dictionary for estimated values 
    selected_values = {'C0':C0_value, 'R0':R0_value,'RQ':RQ_value,\
        'Q factor':Q_value,'Frequency Center':f0_value}
    
    ### Return all possible R/C combinations
    if visible.casefold() == 'on':
        print(tabulate(selected_values,headers='keys'))
        return selected_values
    else:
        return selected_values




In [19]:
from fliege_notch import Notch_Fliege

In [20]:
Notch_Fliege(50, margin=50)

NameError: name 'R1_value' is not defined

In [11]:
def MFB_BandPass(cutoff, gain = 1, Q = 0.707, margin = 2, filter_type = None, cap = None, res = None, visible='Off'):
    ### Required libraries are imported
    import si_prefix
    import numpy as np
    from tabulate import tabulate
    
    #
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]
    
    ### Initialize capacitor values
    if cap is None:
        # Default capacitor values
        cap_p = np.array([0.5, 0.75, 1, 1.2, 1.3, 1.5, 1.6, 1.8, 2, 2.2, 2.4, 2.5, 2.7, 3, 3.3, 3.6, 3.9, 4.3, 4.7, \
         5, 6, 6.2, 6.8, 7, 7.5, 8, 8.2, 9, 10, 11, 12, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 39,\
         43, 47, 51, 56, 62, 68, 82, 100, 110, 120, 150, 180, 200, 220, 330, 470, 680, 820])*(10**-12)

        cap_n = np.array([1, 1.5, 2.2, 3.3, 4.7, 5.6, 6.8, 8.2, 10, 15, 22, 33, 47, 68, 82, 100, 220, 470, 680, 820])\
                         *(10**-9)

        cap_u = np.array([1, 1.5, 2.2, 3.3, 4.7, 5.6])*(10**-6)

        cap_library = np.concatenate((cap_p,cap_n,cap_u),axis=0)
        
    elif type(cap) is np.ndarray:
        # Assign custom capacitor values
        cap_library = cap
        
    else:
        raise TypeError("Capacitor values must be np.ndarray data type!")
    

    ### Initialize resistor values
    if res is None:
        # Default resistor values
        res_0 = np.array([1, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6,\
                          3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1])
        res_1 = res_0*10
        res_2 = res_1*10
        res_3 = res_2*10
        res_4 = res_3*10
        res_5 = res_4*10
        res_6 = res_5*10
        res_library = np.concatenate(([0],res_0,res_1,res_2,res_3,res_4,res_5,res_6,[10**7]),axis=0)
        
    elif type(res) is np.ndarray:
        # Assign custom resistor values
        res_library = res
    
    else:
        raise TypeError("Resistor values must be np.ndarray data type!")
    
    
    ### Filter type and Q value 
    # Available filter types are 'bessel' , 'linear' , 'butter'
    if filter_type is None:
        # No filter type is specified
        # Look for Q value
        Q_ideal = Q
        Q_margin = (margin/100)*Q_ideal
    elif type(filter_type) is not str:
        raise TypeError("filter type must be string!")
    else:
        try:
            filters = {'bessel': 0.577, 'linear': 0.644, 'butter': 0.707}
            Q_ideal = filters[filter_type.casefold()]
            Q_margin = (margin/100)*Q_ideal
        except KeyError:
            print('Filter type must be "Bessel" , "Linear" or "Butter" ')
            raise
    
    ### Assign ideal gain value
    gain_ideal = gain
    
    ### Assign cut-off frequency
    fc_ideal = cutoff
    
    ### Start R and C value estimation
    # wc = 2*np.pi*fc_ideal
    C3_value, C4_value = [], []
    R1_value, R2_value, R5_value = [], [], []
    Q_value, Gain_value, fc_value = [], [], []
    for c_temp in cap_library:
        # Start assigning C3 capacitor value
        C3_candid = c_temp
        k = 2*np.pi*fc_ideal*C3_candid
        H = Q_ideal * gain_ideal
        # Find ideal C1 and C2 values
        C4_ideal = C3_candid
        # C2_ideal = c_temp*gain_ideal
        
        # Find the closest C1 and C2 values from capacitor library
        simple_C3 = si_prefix.split(C4_ideal)
        # simple_C2 = si_prefix.split(C2_ideal)
        if (simple_C3[1] == -6 and simple_C3[0]>= 6) or simple_C3[1] > -6:
            continue
        else:
            C3_candid = find_nearest(cap_library,C4_ideal)
            C4_candid = find_nearest(cap_library,C4_ideal)
        
        # Estimate ideal R1 and R2 values
        R1_ideal = 1 / (H*k)
        R2_ideal = 1 / (np.abs(2*Q_ideal - H)*k)
        R5_ideal = 2 * Q_ideal / k
        
        # Find the closest R1 and R2 values from resistor library
        simple_R1 = si_prefix.split(R1_ideal)
        simple_R2 = si_prefix.split(R2_ideal)
        simple_R5 = si_prefix.split(R5_ideal)
        if (simple_R1[1] == 6 and simple_R1[0] >= 10) or simple_R1[1] > 6:
            continue
        elif (simple_R2[1] == 6 and simple_R2[0] >= 10) or simple_R2[1] > 6:
            continue
        elif (simple_R5[1] == 6 and simple_R5[0] >= 10) or simple_R5[1] > 6:
            continue
        else:
            R1_candid_base = np.around(np.array(simple_R1[0]),decimals=1)
            R1_candid_power = (10**simple_R1[1])
            R1_candid = find_nearest(res_library,R1_candid_base*R1_candid_power)
            
            R2_candid_base = np.around(np.array(simple_R2[0]),decimals=1)
            R2_candid_power = (10**simple_R2[1])        
            R2_candid = find_nearest(res_library,R2_candid_base*R2_candid_power)

            R5_candid_base = np.around(np.array(simple_R5[0]),decimals=1)
            R5_candid_power = (10**simple_R5[1])        
            R5_candid = find_nearest(res_library,R5_candid_base*R5_candid_power)

            Q_candid = R5_candid * k / 2
            # fc_candid = np.sqrt(1/(R1_candid*R2_candid*C2_candid*C3_candid))/(2*np.pi)
            gain_actual = Q_candid / (R1_candid * k)
        
        # Check if actual Q values is between decided margin values
        if np.abs(Q_candid-Q_ideal) <= Q_margin:
            # Append capacitor and resistor values to empty array
            C3_value.append(si_prefix.si_format(C3_candid))
            C4_value.append(si_prefix.si_format(C4_candid))
            R1_value.append(si_prefix.si_format(R1_candid))
            R2_value.append(si_prefix.si_format(R2_candid))
            R5_value.append(si_prefix.si_format(R5_candid))

            Q_value.append(Q_candid)
            Gain_value.append(gain_actual)
            fc_value.append((fc_ideal/Q_candid)/2)
    
    ### Create dictionary for estimated values 
    selected_values = {'C3':C3_value, 'C4':C4_value,'R1':R1_value,'R2':R2_value,'R5':R5_value,\
        'Q factor':Q_value,'Cut-off':fc_value,'Gain':Gain_value}
    
    ### Return all possible R/C combinations
    if visible.casefold() == 'on':
        print(tabulate(selected_values,headers='keys'))
        return selected_values
    else:
        return selected_values


In [12]:
MFB_BandPass(25, gain=40, Q=0.577, visible='On')

C3       C4       R1       R2       R5         Q factor    Cut-off     Gain
-------  -------  -------  -------  -------  ----------  ---------  -------
820.0 p  820.0 p  330.0 k  360.0 k  9.1 M      0.586064    21.3287  13.7879
2.2 n    2.2 n    130.0 k  130.0 k  3.3 M      0.570199    21.9222  12.6923
3.3 n    3.3 n    82.0 k   91.0 k   2.2 M      0.570199    21.9222  13.4146
5.6 n    5.6 n    51.0 k   51.0 k   1.3 M      0.57177     21.8619  12.7451
6.8 n    6.8 n    39.0 k   43.0 k   1.1 M      0.587478    21.2774  14.1026
8.2 n    8.2 n    33.0 k   36.0 k   910.0 k    0.586064    21.3287  13.7879
22.0 n   22.0 n   12.0 k   13.0 k   330.0 k    0.570199    21.9222  13.75
33.0 n   33.0 n   8.2 k    9.1 k    220.0 k    0.570199    21.9222  13.4146
68.0 n   68.0 n   3.9 k    4.3 k    110.0 k    0.587478    21.2774  14.1026
82.0 n   82.0 n   3.3 k    3.6 k    91.0 k     0.586064    21.3287  13.7879
220.0 n  220.0 n  1.3 k    1.3 k    33.0 k     0.570199    21.9222  12.6923
680.0 n  680.0

{'C3': ['820.0 p',
  '2.2 n',
  '3.3 n',
  '5.6 n',
  '6.8 n',
  '8.2 n',
  '22.0 n',
  '33.0 n',
  '68.0 n',
  '82.0 n',
  '220.0 n',
  '680.0 n',
  '820.0 n',
  '2.2 µ',
  '3.3 µ',
  '5.6 µ'],
 'C4': ['820.0 p',
  '2.2 n',
  '3.3 n',
  '5.6 n',
  '6.8 n',
  '8.2 n',
  '22.0 n',
  '33.0 n',
  '68.0 n',
  '82.0 n',
  '220.0 n',
  '680.0 n',
  '820.0 n',
  '2.2 µ',
  '3.3 µ',
  '5.6 µ'],
 'R1': ['330.0 k',
  '130.0 k',
  '82.0 k',
  '51.0 k',
  '39.0 k',
  '33.0 k',
  '12.0 k',
  '8.2 k',
  '3.9 k',
  '3.3 k',
  '1.3 k',
  '390.0 ',
  '330.0 ',
  '130.0 ',
  '82.0 ',
  '51.0 '],
 'R2': ['360.0 k',
  '130.0 k',
  '91.0 k',
  '51.0 k',
  '43.0 k',
  '36.0 k',
  '13.0 k',
  '9.1 k',
  '4.3 k',
  '3.6 k',
  '1.3 k',
  '430.0 ',
  '360.0 ',
  '130.0 ',
  '91.0 ',
  '51.0 '],
 'R5': ['9.1 M',
  '3.3 M',
  '2.2 M',
  '1.3 M',
  '1.1 M',
  '910.0 k',
  '330.0 k',
  '220.0 k',
  '110.0 k',
  '91.0 k',
  '33.0 k',
  '11.0 k',
  '9.1 k',
  '3.3 k',
  '2.2 k',
  '1.3 k'],
 'Q factor': [0.5860641095